In [ ]:
import panel as pn
import numpy as np
import pandas as pd
import holoviews as hv

import itertools
from functools import partial

import time
from panel.widgets import Tqdm

from tornado import gen
from tornado.ioloop import PeriodicCallback

from folium.plugins import HeatMap, HeatMapWithTime, MarkerCluster

from bokeh import models, plotting, io
from bokeh.layouts import column
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, RangeTool, BoxAnnotation

from holoviews import opts
from holoviews.streams import Pipe, Buffer
from holoviews.operation.timeseries import rolling, rolling_outlier_std

import warnings
warnings.filterwarnings("ignore")

hv.extension('bokeh')

%matplotlib inline

<p style= "background-color:#000000;font-family:Georgia;color:#FFFFFF;font-size:500%;text-align:center;border-radius:10px 10px;border-style:solid;border-width:3px;border-color:#000000;"><b>Streaming　Plot</b></p>

- Data Streaming Visualization
- Try to run it in your own environment. Since it is streaming, it will not run on Kaggle.
- データストリーミングVisualization
- 自身の環境で実行してみてください。ストリーミングなのでKaggle上では動作しません。

In [ ]:
hv.renderer('bokeh').theme= 'dark_minimal'

In [ ]:
train = pd.read_csv('D:\JupyterNotebook\WorldWarmingDataset\GlobalLandTemperaturesByCountry.csv')

In [ ]:
train.isnull().sum()

In [ ]:
country_list = ['Canada', 'Brazil', 'Congo', 'India', 'United Kingdom', 'United States', 'Swaziland', 'Italy', 'Europe', 'France', 'Japan', 'Russia']
train['Dates'] = pd.to_datetime(train['dt'])
train['Year'] = train['Dates'].dt.year
train['Month'] = train['Dates'].dt.month

train_country_isin = train[train['Country'].isin(country_list)] # ~ not in
train_copy = train_country_isin.copy()
train_copy.fillna(method='ffill', inplace=True)

In [ ]:
train_copy.isnull().sum()

------
------

In [ ]:
japan_data = train_copy.loc[train_copy['Country'] == 'Japan']

In [ ]:
japan_year_avg = pd.pivot_table(japan_data, values='AverageTemperature', index='Year', aggfunc='mean')
japan_year_avg['10_year_MA'] = japan_year_avg['AverageTemperature'].rolling(10).mean()

-----
# Stream Data
-----

In [ ]:
japan_replace_data = japan_year_avg.reset_index().loc[:, ['Year', 'AverageTemperature']]

In [ ]:
default_frame = pd.DataFrame({'Year': [], 'AverageTemperature': [], 'count': []}, columns=['Year', 'AverageTemperature', 'count'])
count = 0
df_buffer = Buffer(default_frame, length=100, index=False)

In [ ]:
def add_plot():
    while True:
        x, y, count = 0, 0, 0
        for i in range(len(japan_replace_data.reset_index()['Year'])):
            x = japan_replace_data.reset_index()['Year'][i] # += 
            y = japan_replace_data.reset_index()['AverageTemperature'][i] # += 
            count += 1
            df_data = pd.DataFrame([(x, y, count)], columns=['Year', 'AverageTemperature', 'count'])
            if len(japan_replace_data.reset_index()['Year']) != count:
                yield df_data
            else:
                df_buffer.clear()
                del df_data, count

@gen.coroutine
def f():
    df_buffer.send(next(add_p))

In [ ]:
add_p = add_plot()

In [ ]:
asdasd

In [ ]:
cb_d = PeriodicCallback(f, 100)
cb_d.start()


point_plt = hv.DynamicMap(hv.Points, streams=[df_buffer])

line_plt = hv.DynamicMap(hv.Curve, streams=[df_buffer]).opts(padding=0.1, width=600, show_grid=True) # one + two : line & point

table_plt = hv.DynamicMap(hv.Table, streams=[df_buffer])

hist_dmap = hv.DynamicMap(hv.Dataset, streams=[df_buffer])
hist_plt = hv.operation.histogram(hist_dmap, dimension='AverageTemperature')

box_plt = hv.DynamicMap(partial(hv.BoxWhisker, kdims=[], vdims='AverageTemperature'), streams=[df_buffer])


#table_plt + point_plt * line_plt

In [ ]:
gspec = pn.GridSpec(width=1200, height=650)

gspec[0,   0  ] = table_plt
gspec[0,   1:5] = line_plt
gspec[1,   2:3] = point_plt
gspec[1,   3:5] = box_plt
gspec[1,   0:2] = hist_plt

gspec

In [ ]:
cb_d.stop()